In [9]:
import pickle
import pandas as pd
import multiprocessing as mp
import numpy as np
import pickle
import matplotlib.pyplot as plt
from Bio import SeqIO
import os
import subprocess
import ast
import sys
sys.path.insert(0, '/n/home11/rkapoor')
import tax_pkg
from tax_pkg import taxid
from tax_pkg import accession2taxid
from collections import Counter

## Consolidate final list of chimeric HGT candidates from manual tree annotation

In [18]:
#load csv with manual annotations of  origin for confirmed hgt and metazoan intervals from chimeras
metadf=pd.read_csv("meta_origin_final.csv",index_col=0)
metas=metadf.index
hgtdf=pd.read_csv("hgt_origin_final.csv",index_col=0)
hgtdf["gene"]=[x.split(";")[1] for x in hgtdf.index]

gene_genome={x.split(";")[1]:x.split(";")[0] for x in hgtdf.index}
gene_organism={x:y for x,y in zip(hgtdf.gene,hgtdf.organism)}
hgts=hgtdf.index

#final set of chimeric hgt genes (both confirmed HGT and metazoan intervals)
genes=set([x.split(";")[1] for x in  hgts])&set([x.split(";")[1] for x in metas])
metas_2=[x for x in metas if x.split(";")[1] in genes]
hgts_2=[x for x in hgts if x.split(";")[1] in genes]


## Make a dataframe storing data (accession, description, taxonomic distribution) for each chimera

In [22]:
direct="/n/holyscratch01/extavour_lab/Lab/rkapoor/"

In [24]:
##add source genome and organism 
df_prot=pd.DataFrame(index=list(set(genes)))
df_prot.loc[:,"genome"]=[gene_genome[x] for x in df_prot.index]
df_prot.loc[:,"organism"]=[gene_organism[x] for x in df_prot.index]

In [27]:
##add protein description 
for genome in set(df_prot.genome):
    file_path=f"/n/holyscratch01/extavour_lab/Lab/rkapoor/ncbi_dataset/data/{genome}/genomic.gff"
    column_names = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    dfg = pd.read_csv(file_path, sep='\t', comment='#', names=column_names)
    for index, row in df_prot[df_prot.genome==genome].iterrows():
        try:
            dfi=dfg[dfg.attributes.str.contains(index)]
            df_prot.loc[index,"description"]=list(dfi.attributes)[0].split(";product=")[1].split(";")[0]
        except:
            print(genome,index)

In [28]:
##add information on the taxonomic range of all secondary chimeras to the dataframe
secmap={}
speciesmap={}
for prot in df_prot.index:
    td=list(set([x for x in hgts_2 if prot in x])|set([x for x in metas_2 if prot in x]))
    target_set=[]
    species_set=[]
    for t in td:
        dft=pd.read_csv(f"/n/holylabs/LABS/extavour_lab/Users/rkapoor/hmmer_phylo_data/{t}/secondary_chimera.tsv",sep="\t")
        target_set.append(set(dft.target_name))
        species_set.append(set(dft.species))
    
    dft=pd.read_csv(f"/n/holylabs/LABS/extavour_lab/Users/rkapoor/hmmer_phylo_data/{t}/phylo_tax2.tsv",sep="\t")
    target_set = set.intersection(*target_set)
    species_set = set.intersection(*species_set)
    df_prot.loc[prot,"taxid_set"]=str(set(dft[dft.target_name.isin(target_set)].taxid))
    df_prot.loc[prot,"species_set"]=str(species_set)
    df_prot.loc[prot,"n_species"]=len(species_set)
    df_prot.loc[prot,"phylum_dist"]=str(dict(Counter(dft[dft.target_name.isin(target_set)].phylum)))
    df_prot.loc[prot,"order_dist"]=str(dict(Counter(dft[dft.target_name.isin(target_set)].order)))
    secmap[prot]=set(dft.target_name)
    speciesmap[prot]=set(dft.species)
    
for prot in df_prot.index:
    classes=[]
    for tid in ast.literal_eval(df_prot.loc[prot,"taxid_set"]):
        l=taxid.get_lineage(tid,{})
        classes.append(taxid.get_class(tid,l))
    df_prot.loc[prot,"class_dist"]=str(dict(Counter(classes)))

## Determine the taxonomic span of each HGT-chimera
Tax span is the lowest NCBI taxonomic rank encompassing all primary and secondary chimeras for a given HGT-chimera

In [ ]:
#load ncbi taxonomy datatabase 
df_tax=pd.read_csv("/n/holyscratch01/extavour_lab/Lab/rkapoor/dbs/names.dmp",sep="\t",header=None,index_col=0)
#ordered list of ncbi taxonomic ranks (highest to smallest)
tax_ranks=['superkingdom', 'kingdom', 'subkingdom', 'superphylum', 'phylum', 'subphylum', 'infraphylum', 'superclass', 'class', 'subclass', 'infraclass', 'cohort', 'subcohort', 'superorder', 'order', 'suborder', 'infraorder', 'parvorder', 'superfamily', 'family', 'subfamily', 'tribe', 'subtribe', 'genus', 'subgenus', 'section', 'subsection', 'series', 'subseries', 'species group', 'species subgroup', 'species', 'forma specialis', 'subspecies', 'varietas', 'subvariety', 'forma', 'serogroup', 'serotype', 'strain', 'isolate']
for prot in df_prot.index:
    # for every protein, determine the lowest taxonomic rank encompassing all chimeras
    lineages=[]
    for tid in ast.literal_eval(df_prot.loc[prot,"taxid_set"]):
        lineages.append(taxid.get_lineage(tid,{}))
    keys=[set(x.keys()) for x in lineages]
    keys=set.intersection(*keys)-set("no rank")
    keys=[x for x in tax_ranks if x in keys][::-1]
    for k in keys:
        tax_vals=set([str(l[k]) for l in lineages])

        if len(set(tax_vals))==1:
            df_prot.loc[prot,"tax_span"]=list(set(tax_vals))[0]
            df_prot.loc[prot,"tax_span_rank"]=k
            break

## Cluster HGT-Chimeras into orthologous groups using a network approach
Network is constructed such that edges connect any two HGT-chimeras related such that at least one is a secondary chimera of the other. Orthologous clusters likely reflecting a single origin are defiend as weakly connected components in the network.

In [32]:
import networkx as nx

# Define the directed adjacency list
adjacency_list = secmap

# Create a directed graph from the adjacency list
graph = nx.DiGraph(adjacency_list)

# Find connected components
components = nx.weakly_connected_components(graph)
weak=[]
# Print the connected components
for component in components:
    weak.append(component)

components = nx.strongly_connected_components(graph)
strong=[]
# Print the connected components
for component in components:
    strong.append(component)


In [35]:
#insert weak cluster identifiers into protein dataframe 
map1={x:y for x,y in zip(list(range(len(weak))),weak)}
map2={}
for x in map1:
    for y in map1[x]:
        map2[y]=x
for index, row in df_prot.iterrows():
    df_prot.loc[index,"weak_cc"]=map2[index]

## Make a dataframe for each orthologous cluster, with taxonomic information merged for all constituent chimeras within the cluster
Note that clustering was observed to not alter the total taxonomic span/rank of each HGT-chimera, so this information is simply copied from the protein df

In [22]:
df_cluster=pd.DataFrame(index=list(set(df_prot.weak_cc)),columns=["proteins"]+list(df_prot.columns))

In [24]:
weak_cc_counts=dict(Counter(df_prot.weak_cc))
multiple_ccs=[x for x in weak_cc_counts if weak_cc_counts[x]>1]
for x in weak_cc_counts:
    df_cluster.loc[x,"proteins"]=str(list(df_prot[df_prot.weak_cc==x].index))
    if weak_cc_counts[x]==1:
        
        df_cluster.loc[x,list(df_prot.columns)]=list(df_prot[df_prot.weak_cc==x].iloc[0,:])
        
        
    else:
        df_cluster.loc[x,list(df_prot.columns)]=list(df_prot[df_prot.weak_cc==x].iloc[0,:])
       
        species_sets=[ast.literal_eval(df_prot.loc[xi,"species_set"]) for xi in df_prot[df_prot.weak_cc==x].index]
        
        species_sets=set.union(*species_sets)
        df_cluster.loc[x,"n_species"]=len(species_sets)
        df_cluster.loc[x,"species_set"]=str(species_sets)
        taxid_sets=[ast.literal_eval(df_prot.loc[xi,"taxid_set"]) for xi in df_prot[df_prot.weak_cc==x].index]
        taxid_sets=set.union(*taxid_sets)
        
        
        sec_sets=[ast.literal_eval(df_prot.loc[xi,"secondary_chimeras"]) for xi in df_prot[df_prot.weak_cc==x].index]
        sec_sets=set.union(*sec_sets)
        df_cluster.loc[x,"secondary_chimeras"]=str(sec_sets)
df_cluster.to_csv("cluster_info.tsv",sep="\t")